# Classes

## Caregiver

In [9]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime
import calendar

class Caregiver:
    def __init__(self, name: str, phone: str, email: str, is_paid: bool = False):
        self.name = name
        self.phone = phone
        self.email = email
        self.is_paid = is_paid
        self.pay_rate = 20.0 if is_paid else 0.0
        self.hours = 0
        self.availability = self._create_default_availability()
        
    def _create_default_availability(self) -> dict:
        """Creates default weekly availability schedule with random preferences"""
        import random
        
        schedule = {}
        # For each day of the week (0 = Monday, 6 = Sunday)
        for day in range(7):
            schedule[day] = {
                'AM': 'available',  # Default to available initially
                'PM': 'available'
            }
            
            # Randomly assign preferences/unavailability
            # Weights make 'available' most common, then 'preferred', then 'unavailable'
            schedule[day]['AM'] = random.choices(
                ['preferred', 'available', 'unavailable'], 
                weights=[0.3, 0.5, 0.2]
            )[0]
            schedule[day]['PM'] = random.choices(
                ['preferred', 'available', 'unavailable'], 
                weights=[0.3, 0.5, 0.2]
            )[0]
        
        return schedule
        
    @classmethod
    def ensure_coverage(cls, caregivers: list):
        """Ensures at least one caregiver is available for each time slot"""
        for day in range(7):
            for shift in ['AM', 'PM']:
                # Check if anyone is available or preferred for this slot
                available = False
                for caregiver in caregivers:
                    if caregiver.availability[day][shift] in ['available', 'preferred']:
                        available = True
                        break
                
                # If no one is available, randomly select a caregiver to be available
                if not available:
                    random_caregiver = random.choice(caregivers)
                    random_caregiver.availability[day][shift] = 'available'

    def set_availability(self, day: int, shift: str, status: str):
        """Set availability for a specific day and shift"""
        if status not in ['preferred', 'available', 'unavailable']:
            raise ValueError("Status must be 'preferred', 'available', or 'unavailable'")
        self.availability[day][shift] = status
        
    def add_hours(self, hours: float):
        """Add worked hours"""
        self.hours += hours



## Schedule

In [10]:

class Schedule:
    def __init__(self, year: int, month: int):
        self.year = year
        self.month = month
        self.caregivers = []
        self.schedule = {}
        
    def add_caregiver(self, caregiver: Caregiver):
        self.caregivers.append(caregiver)
        
    def generate_schedule(self):
        """Generate monthly schedule based on caregiver availability"""
        _, num_days = calendar.monthrange(self.year, self.month)
        
        for day in range(1, num_days + 1):
            weekday = calendar.weekday(self.year, self.month, day)
            
            # Find available caregivers for each shift
            am_shift = self._assign_shift(weekday, 'AM')
            pm_shift = self._assign_shift(weekday, 'PM')
            
            self.schedule[day] = {
                'AM': am_shift,
                'PM': pm_shift
            }
            
            # Add hours for paid caregivers
            if am_shift and am_shift.is_paid:
                am_shift.add_hours(6)
            if pm_shift and pm_shift.is_paid:
                pm_shift.add_hours(6)
    
    def _assign_shift(self, weekday: int, shift: str) -> Caregiver:
        """Assign a caregiver to a shift based on availability"""
        preferred = [c for c in self.caregivers if c.availability[weekday][shift] == 'preferred']
        available = [c for c in self.caregivers if c.availability[weekday][shift] == 'available']
        
        if preferred:
            return min(preferred, key=lambda x: x.hours)
        elif available:
            return min(available, key=lambda x: x.hours)
        return None

    def generate_html_calendar(self):
        """Generate HTML calendar with schedule"""
        html = f"""
        <html>
        <head>
            <title>Care Schedule - {calendar.month_name[self.month]} {self.year}</title>
            <style>
                table {{ border-collapse: collapse; width: 100%; }}
                th, td {{ border: 1px solid black; padding: 10px; text-align: center; }}
                th {{ background-color: #f2f2f2; }}
            </style>
        </head>
        <body>
            <h1>Care Schedule - {calendar.month_name[self.month]} {self.year}</h1>
            <table>
                <tr>
                    <th>Mon</th><th>Tue</th><th>Wed</th><th>Thu</th><th>Fri</th><th>Sat</th><th>Sun</th>
                </tr>
        """
        
        first_day = calendar.weekday(self.year, self.month, 1)
        _, num_days = calendar.monthrange(self.year, self.month)
        
        current_day = 1
        html += "<tr>"
        
        for _ in range(first_day):
            html += "<td></td>"
            
        while current_day <= num_days:
            if (first_day + current_day - 1) % 7 == 0 and current_day != 1:
                html += "</tr><tr>"
                
            day_schedule = self.schedule.get(current_day, {'AM': None, 'PM': None})
            am_name = day_schedule['AM'].name if day_schedule['AM'] else "OPEN"
            pm_name = day_schedule['PM'].name if day_schedule['PM'] else "OPEN"
            
            html += f"""
                <td>
                    {current_day}<br>
                    <b>AM:</b> {am_name}<br>
                    <b>PM:</b> {pm_name}
                </td>
            """
            
            current_day += 1
            
        while (first_day + current_day - 1) % 7 != 0:
            html += "<td></td>"
            current_day += 1
            
        html += """
                </tr>
            </table>
        </body>
        </html>
        """
        
        with open(f"care_schedule_{self.year}_{self.month}.html", "w") as f:
            f.write(html)

def generate_pay_report(schedule: Schedule):
    """Generate weekly pay report for paid caregivers"""
    report = f"Pay Report for {calendar.month_name[schedule.month]} {schedule.year}\n"
    report += "=" * 50 + "\n\n"
    
    paid_caregivers = [c for c in schedule.caregivers if c.is_paid]
    total_monthly_pay = 0
    
    for caregiver in paid_caregivers:
        weekly_hours = caregiver.hours / 4  # Approximate weekly hours
        weekly_pay = weekly_hours * caregiver.pay_rate
        monthly_pay = weekly_pay * 4
        
        report += f"Caregiver: {caregiver.name}\n"
        report += f"Weekly Hours: {weekly_hours:.2f}\n"
        report += f"Weekly Pay: ${weekly_pay:.2f}\n"
        report += f"Monthly Pay: ${monthly_pay:.2f}\n"
        report += "-" * 30 + "\n"
        
        total_monthly_pay += monthly_pay
    
    report += f"\nTotal Monthly Payroll: ${total_monthly_pay:.2f}"
    
    with open(f"pay_report_{schedule.month}_{schedule.year}.txt", "w") as f:
        f.write(report)


## CareScheduleGUI

In [11]:

class CareSchedulerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Care Schedule Generator")
        self.root.geometry("1200x1000")
        
        # Initialize caregivers list first
        self.caregivers = [
            Caregiver("Alice Smith", "555-0101", "alice@email.com", True),
            Caregiver("Bob Jones", "555-0102", "bob@email.com", True),
            Caregiver("Carol Wilson", "555-0103", "carol@email.com", False),
            Caregiver("David Brown", "555-0104", "david@email.com", True),
            Caregiver("Eve Davis", "555-0105", "eve@email.com", False),
            Caregiver("Frank Miller", "555-0106", "frank@email.com", True),
            Caregiver("Grace Taylor", "555-0107", "grace@email.com", False),
            Caregiver("Henry White", "555-0108", "henry@email.com", True)
        ]
        
        # Ensure coverage after random availability is generated
        Caregiver.ensure_coverage(self.caregivers)

        # Store caregiver availability data
        self.availability_data = {}
        
        # Create main containers
        self.create_header_frame()
        self.create_date_frame()
        self.create_caregiver_frame()
        self.create_buttons_frame()

    def create_header_frame(self):
        """Create the header section"""
        header_frame = ttk.Frame(self.root, padding="10")
        header_frame.grid(row=0, column=0, sticky=(tk.W, tk.E))
        
        ttk.Label(header_frame, 
                 text="Care Schedule Generator", 
                 font=('Helvetica', 16, 'bold')).grid(row=0, column=0)

    def create_date_frame(self):
        """Create the date selection section"""
        date_frame = ttk.LabelFrame(self.root, text="Schedule Period", padding="20")  # Increased padding
        date_frame.grid(row=1, column=0, padx=20, pady=10, sticky=(tk.W, tk.E))  
        
        # Year selection with extended range
        current_year = datetime.now().year
        ttk.Label(date_frame, text="Year:").grid(row=0, column=0, padx=5)
        self.year_var = tk.StringVar(value=str(current_year))
        year_range = [str(year) for year in range(current_year - 1, current_year + 4)]
        year_combo = ttk.Combobox(date_frame, 
                                textvariable=self.year_var, 
                                values=year_range,
                                width=10,
                                state='readonly')
        year_combo.grid(row=0, column=1, padx=5)
        
        # Month selection
        ttk.Label(date_frame, text="Month:").grid(row=0, column=2, padx=5)
        self.month_var = tk.StringVar(value=calendar.month_name[datetime.now().month])
        month_combo = ttk.Combobox(date_frame, 
                                textvariable=self.month_var,
                                values=[calendar.month_name[i] for i in range(1, 13)],
                                width=10,
                                state='readonly')
        month_combo.grid(row=0, column=3, padx=5)

    def create_caregiver_frame(self):
        """Create the caregiver availability section"""
        # Make the column expand to use full width
        self.root.grid_columnconfigure(0, weight=1)
        self.root.grid_rowconfigure(2, weight=1)  # Row 2 is where caregiver_frame is

        caregiver_frame = ttk.LabelFrame(self.root, text="Caregiver Availability", padding="20")
        caregiver_frame.grid(row=2, column=0, padx=20, pady=10, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Configure the caregiver_frame to expand both horizontally and vertically
        caregiver_frame.grid_columnconfigure(0, weight=1)
        caregiver_frame.grid_rowconfigure(0, weight=1)
        
        # Define colors for different availability states
        availability_colors = {
            'preferred': '#90EE90',  # Light green
            'available': '#87CEEB',  # Light blue
            'unavailable': '#FFB6C1'  # Light red
        }

        # Make the canvas expand to fill the frame
        canvas = tk.Canvas(caregiver_frame)
        scrollbar = ttk.Scrollbar(caregiver_frame, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)

        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )

        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)

        # Make the canvas expand to fill available space
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        # Create the availability grid
        # Headers for days
        days = ['Caregiver', 'Shift'] + list(calendar.day_name)
        for col, day in enumerate(days):
            width = 6 if col == 1 else 12  # Make Shift column narrower
            anchor = "w" if col == 0 else "center"  # Left align Caregiver column
            ttk.Label(scrollable_frame, 
                    text=day, 
                    font=('Helvetica', 10, 'bold'),
                    width=width,
                    anchor=anchor).grid(row=0, column=col, padx=2, pady=5)

        def am_color_update(menu, var):
            menu.configure(bg=availability_colors[var.get()])

        def pm_color_update(menu, var):
            menu.configure(bg=availability_colors[var.get()])
        
        # Create rows for each caregiver
        current_row = 1
        for caregiver in self.caregivers:
            # Add more padding to top of caregiver name if not first caregiver
            top_pady = 15 if current_row > 1 else 2
            
            # Caregiver name spanning both AM and PM rows
            ttk.Label(scrollable_frame, 
                    text=caregiver.name,
                    width=15).grid(row=current_row, column=0, rowspan=2, padx=2, pady=(top_pady,2))
            
            # AM shift
            ttk.Label(scrollable_frame, 
                    text="AM",
                    width=8).grid(row=current_row, column=1, padx=2, pady=(top_pady,1))
            
            # PM shift
            ttk.Label(scrollable_frame, 
                    text="PM",
                    width=8).grid(row=current_row + 1, column=1, padx=2, pady=(0,2))

            # Availability comboboxes for each day
            for day_idx, day in enumerate(calendar.day_name):
                day_index = list(calendar.day_name).index(day)
            
                # AM shift menu
                am_value = caregiver.availability[day_index]['AM']
                am_var = tk.StringVar(value=am_value)
              
                am_menu = tk.OptionMenu(scrollable_frame, am_var,
                                      'preferred', 'available', 'unavailable')
                am_menu.configure(width=10, bg=availability_colors[am_value])
                am_menu.grid(row=current_row, column=day_idx + 2, padx=2, pady=(top_pady,1))

                am_var.trace_add('write', lambda *args, 
                    menu=am_menu, var=am_var: 
                    am_color_update(menu, var))

                # PM shift menu  
                pm_value = caregiver.availability[day_index]['PM']
                pm_var = tk.StringVar(value=pm_value)

                pm_menu = tk.OptionMenu(scrollable_frame, pm_var,
                                      'preferred', 'available', 'unavailable')
                pm_menu.configure(width=10, bg=availability_colors[pm_value])
                pm_menu.grid(row=current_row + 1, column=day_idx + 2, padx=2, pady=(0,2))

                pm_var.trace_add('write', lambda *args, 
                    menu=pm_menu, var=pm_var: 
                    pm_color_update(menu, var))
                
                # Store the variables
                key = (day, caregiver.name)
                self.availability_data[key] = {'AM': am_var, 'PM': pm_var}
            current_row += 2  # Move to next caregiver (2 rows per caregiver)


        # Pack the canvas and scrollbar in the container
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

    def create_day_schedule(self, frame, day):
        """Create the schedule grid for a single day"""
        # Headers
        ttk.Label(frame, text="Caregiver", font=('Helvetica', 10, 'bold')).grid(row=0, column=0, padx=5)
        ttk.Label(frame, text="Morning (7AM-1PM)", font=('Helvetica', 10, 'bold')).grid(row=0, column=1, padx=5)
        ttk.Label(frame, text="Afternoon (1PM-7PM)", font=('Helvetica', 10, 'bold')).grid(row=0, column=2, padx=5)
        
        # Create entries for each caregiver
        for i, caregiver in enumerate(self.caregivers, 1):
            ttk.Label(frame, text=caregiver.name).grid(row=i, column=0, padx=5, pady=2)
            
            # Morning availability
            am_var = tk.StringVar(value='available')
            am_combo = ttk.Combobox(frame, 
                                  textvariable=am_var,
                                  values=['preferred', 'available', 'unavailable'],
                                  width=15,
                                  state='readonly')
            am_combo.grid(row=i, column=1, padx=5, pady=2)
            
            # Afternoon availability
            pm_var = tk.StringVar(value='available')
            pm_combo = ttk.Combobox(frame, 
                                  textvariable=pm_var,
                                  values=['preferred', 'available', 'unavailable'],
                                  width=15,
                                  state='readonly')
            pm_combo.grid(row=i, column=2, padx=5, pady=2)
            
            # Store the variables
            key = (day, caregiver.name)
            self.availability_data[key] = {'AM': am_var, 'PM': pm_var}

    def create_buttons_frame(self):
        """Create the buttons section"""
        buttons_frame = ttk.Frame(self.root, padding="20")  # Increased padding
        buttons_frame.grid(row=3, column=0, pady=20, sticky=(tk.W, tk.E))  # Increased padding
        
        # Center the buttons
        buttons_frame.grid_columnconfigure(0, weight=1)
        buttons_frame.grid_columnconfigure(5, weight=1)
        
        # Manage Caregivers button
        ttk.Button(buttons_frame, 
                text="Manage Caregivers",
                command=self.create_caregiver_management_window,
                width=20).grid(row=0, column=1, padx=5)
        
        # Generate Schedule button
        ttk.Button(buttons_frame, 
                text="Generate Schedule",
                command=self.generate_schedule,
                width=20).grid(row=0, column=2, padx=5)
        
        # Generate Pay Report button
        ttk.Button(buttons_frame,
                text="Generate Pay Report",
                command=self.generate_pay_report,
                width=20).grid(row=0, column=3, padx=5)
        
        # Exit button
        ttk.Button(buttons_frame,
                text="Exit",
                command=self.root.quit,
                width=20).grid(row=0, column=4, padx=5)

    def generate_schedule(self):
        """Generate the schedule based on input"""
        try:
            # Get selected year and month
            year = int(self.year_var.get())
            month = list(calendar.month_name).index(self.month_var.get())
            
            # Update caregiver availability based on GUI input
            for caregiver in self.caregivers:
                for day_idx, day in enumerate(calendar.day_name):
                    key = (day, caregiver.name)
                    if key in self.availability_data:
                        am_status = self.availability_data[key]['AM'].get()
                        pm_status = self.availability_data[key]['PM'].get()
                        caregiver.set_availability(day_idx, 'AM', am_status)
                        caregiver.set_availability(day_idx, 'PM', pm_status)
            
            # Create and generate schedule
            self.current_schedule = Schedule(year, month)
            for caregiver in self.caregivers:
                self.current_schedule.add_caregiver(caregiver)
            
            self.current_schedule.generate_schedule()
            self.current_schedule.generate_html_calendar()
            
            messagebox.showinfo("Success", 
                            f"Schedule generated successfully for {calendar.month_name[month]} {year}!\n\n"
                            f"File created:\n"
                            f"- care_schedule_{year}_{month}.html")
            
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while generating schedule: {str(e)}")

    def generate_pay_report(self):
        """Generate the pay report"""
        try:
            if not hasattr(self, 'current_schedule'):
                messagebox.showerror("Error", "Please generate a schedule first before creating a pay report.")
                return
                
            generate_pay_report(self.current_schedule)
            
            month = list(calendar.month_name).index(self.month_var.get())
            year = int(self.year_var.get())
            
            messagebox.showinfo("Success", 
                            f"Pay report generated successfully for {calendar.month_name[month]} {year}!\n\n"
                            f"File created:\n"
                            f"- pay_report_{month}_{year}.txt")
            
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while generating pay report: {str(e)}")


    def create_caregiver_management_window(self):
        """Create a new window for caregiver management"""
        self.caregiver_window = tk.Toplevel(self.root)
        self.caregiver_window.title("Caregiver Management")
        self.caregiver_window.geometry("800x600")

        # Bind to window closing
        self.caregiver_window.protocol("WM_DELETE_WINDOW", self.on_caregiver_window_close)
        
        # Create frames
        info_frame = ttk.LabelFrame(self.caregiver_window, text="Caregiver Information", padding="10")
        info_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # Create Treeview
        columns = ('Name', 'Phone', 'Email', 'Status', 'Pay Rate', 'Hours')
        self.caregiver_tree = ttk.Treeview(info_frame, columns=columns, show='headings')
        
        # Define column headings
        for col in columns:
            self.caregiver_tree.heading(col, text=col)
            self.caregiver_tree.column(col, width=100)
        
        # Add scrollbar
        scrollbar = ttk.Scrollbar(info_frame, orient=tk.VERTICAL, command=self.caregiver_tree.yview)
        self.caregiver_tree.configure(yscrollcommand=scrollbar.set)
        
        # Pack Treeview and scrollbar
        self.caregiver_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Add buttons frame
        button_frame = ttk.Frame(self.caregiver_window, padding="10")
        button_frame.pack(fill=tk.X, padx=10, pady=5)
        
        # Add buttons
        ttk.Button(button_frame, text="Add Caregiver", command=self.show_add_caregiver_dialog).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Edit Caregiver", command=self.show_edit_caregiver_dialog).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Delete Caregiver", command=self.delete_caregiver).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Refresh", command=self.refresh_caregiver_list).pack(side=tk.LEFT, padx=5)
        
        # Populate the treeview
        self.refresh_caregiver_list()

    def on_caregiver_window_close(self):
        """Handle caregiver window closing"""
        self.refresh_availability_grid()
        self.caregiver_window.destroy()

    def refresh_availability_grid(self):
        """Refresh the availability grid when caregivers are updated"""
        # Remove existing frame
        for widget in self.root.grid_slaves():
            if isinstance(widget, ttk.LabelFrame) and widget.cget("text") == "Caregiver Availability":
                widget.destroy()
        
        # Recreate the frame
        self.create_caregiver_frame()



    def refresh_caregiver_list(self):
        """Refresh the caregiver list in the treeview"""
        # Clear existing items
        for item in self.caregiver_tree.get_children():
            self.caregiver_tree.delete(item)
        
        # Add all caregivers
        for caregiver in self.caregivers:
            status = "Paid" if caregiver.is_paid else "Family"
            self.caregiver_tree.insert('', tk.END, values=(
                caregiver.name,
                caregiver.phone,
                caregiver.email,
                status,
                f"${caregiver.pay_rate:.2f}/hr" if caregiver.is_paid else "N/A",
                f"{caregiver.hours:.1f}"
            ))

    def show_add_caregiver_dialog(self):
        """Show dialog to add a new caregiver"""
        dialog = tk.Toplevel(self.caregiver_window)
        dialog.title("Add Caregiver")
        dialog.geometry("400x300")
        
        # Create entry fields
        ttk.Label(dialog, text="Name:").grid(row=0, column=0, padx=5, pady=5)
        name_var = tk.StringVar()
        ttk.Entry(dialog, textvariable=name_var).grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(dialog, text="Phone:").grid(row=1, column=0, padx=5, pady=5)
        phone_var = tk.StringVar()
        ttk.Entry(dialog, textvariable=phone_var).grid(row=1, column=1, padx=5, pady=5)
        
        ttk.Label(dialog, text="Email:").grid(row=2, column=0, padx=5, pady=5)
        email_var = tk.StringVar()
        ttk.Entry(dialog, textvariable=email_var).grid(row=2, column=1, padx=5, pady=5)
        
        ttk.Label(dialog, text="Status:").grid(row=3, column=0, padx=5, pady=5)
        is_paid_var = tk.BooleanVar()
        ttk.Checkbutton(dialog, text="Paid Caregiver", variable=is_paid_var).grid(row=3, column=1, padx=5, pady=5)
        
        def save_caregiver(self):
            try:
                new_caregiver = Caregiver(
                    name_var.get(),
                    phone_var.get(),
                    email_var.get(),
                    is_paid_var.get()
                )
                self.caregivers.append(new_caregiver)
                self.refresh_caregiver_list()
                self.refresh_availability_grid()
                dialog.destroy()
                messagebox.showinfo("Success", "Caregiver added successfully!")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to add caregiver: {str(e)}")
                
                ttk.Button(dialog, text="Save", command=save_caregiver).grid(row=4, column=0, columnspan=2, pady=20)

    def show_edit_caregiver_dialog(self):
        """Show dialog to edit selected caregiver"""
        selected_items = self.caregiver_tree.selection()
        if not selected_items:
            messagebox.showwarning("No Selection", "Please select a caregiver to edit.")
            return
        
        # Get selected caregiver
        selected_values = self.caregiver_tree.item(selected_items[0])['values']
        caregiver = next((c for c in self.caregivers if c.name == selected_values[0]), None)
        
        if not caregiver:
            messagebox.showerror("Error", "Caregiver not found.")
            return
        
        dialog = tk.Toplevel(self.caregiver_window)
        dialog.title("Edit Caregiver")
        dialog.geometry("400x300")
        
        # Create entry fields with current values
        ttk.Label(dialog, text="Name:").grid(row=0, column=0, padx=5, pady=5)
        name_var = tk.StringVar(value=caregiver.name)
        ttk.Entry(dialog, textvariable=name_var).grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(dialog, text="Phone:").grid(row=1, column=0, padx=5, pady=5)
        phone_var = tk.StringVar(value=caregiver.phone)
        ttk.Entry(dialog, textvariable=phone_var).grid(row=1, column=1, padx=5, pady=5)
        
        ttk.Label(dialog, text="Email:").grid(row=2, column=0, padx=5, pady=5)
        email_var = tk.StringVar(value=caregiver.email)
        ttk.Entry(dialog, textvariable=email_var).grid(row=2, column=1, padx=5, pady=5)
        
        ttk.Label(dialog, text="Status:").grid(row=3, column=0, padx=5, pady=5)
        is_paid_var = tk.BooleanVar(value=caregiver.is_paid)
        ttk.Checkbutton(dialog, text="Paid Caregiver", variable=is_paid_var).grid(row=3, column=1, padx=5, pady=5)
        
        def update_caregiver(self):
            try:
                caregiver.name = name_var.get()
                caregiver.phone = phone_var.get()
                caregiver.email = email_var.get()
                caregiver.is_paid = is_paid_var.get()
                caregiver.pay_rate = 20.0 if caregiver.is_paid else 0.0
                self.refresh_caregiver_list()
                self.refresh_availability_grid()
                dialog.destroy()
                messagebox.showinfo("Success", "Caregiver updated successfully!")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to update caregiver: {str(e)}")

        
        ttk.Button(dialog, text="Update", command=update_caregiver).grid(row=4, column=0, columnspan=2, pady=20)

    def delete_caregiver(self):
        """Delete selected caregiver"""
        selected_items = self.caregiver_tree.selection()
        if not selected_items:
            messagebox.showwarning("No Selection", "Please select a caregiver to delete.")
            return
        
        if messagebox.askyesno("Confirm Delete", "Are you sure you want to delete this caregiver?"):
            selected_values = self.caregiver_tree.item(selected_items[0])['values']
            self.caregivers = [c for c in self.caregivers if c.name != selected_values[0]]
            self.refresh_caregiver_list()
            self.refresh_availability_grid()
            messagebox.showinfo("Success", "Caregiver deleted successfully!")

# Main

In [12]:
def main():
    root = tk.Tk()
    app = CareSchedulerGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
TypeError: CareSchedulerGUI.show_edit_caregiver_dialog.<locals>.update_caregiver() missing 1 required positional argument: 'self'
